# Recitation Week 8

### Closures, Functions, and Currying

## Currying

We will first study currying in Scala. 
  - Currying named after mathematician/computer scientist Haskell Curry.
  - Multiple argument functions can be seen as a sequence of single argument functions.

More on currying in Scala: https://docs.scala-lang.org/tour/multiple-parameter-lists.html#example

Currying can be useful for many things, including partially applying functions (setting only some of the function parameters).
As we see below, we can't really partially apply 'normal' function definitions:

In [ ]:
// Multiple argument functions
def addTwo (x: Int, y: Int): Int = x + y

val x = addTwo(10) // Error

cmd0.sc:3: not enough arguments for method addTwo: (x: Int, y: Int): Int.
Unspecified value parameter y.
val x = addTwo(10) // Error
              ^Compilation Failed

: 

In [1]:
// Scala supports curried functions: sort of.
def curriedAddTwo (x: Int) (y: Int) = x + y 

defined function curriedAddTwo

Note the type of curriedAddTwo is `Int => (Int => Int)`.

  - If you give it a number, it returns a function from number to number.

In [2]:
val f1 : Int => Int = curriedAddTwo(15)

f1: Int => Int = ammonite.$sess.cmd1$Helper$$Lambda$1929/0x00000008409f3840@38df63b1

In [3]:
val f1 = curriedAddTwo (15)(_)
// f1 has plugged in x but y has not be set. Remember _ makes it an anonymous function
val x = f1(20)
val y = f1(30)
val z = f1(45)

f1: Int => Int = ammonite.$sess.cmd2$Helper$$Lambda$1960/0x0000000840a20840@26b14675
x: Int = 35
y: Int = 45
z: Int = 60

In [4]:
// we could also call curriedAddTwo directly, if we don't want to fix x to one value
// this might remind you of foldLeft, an example of a curried function we have been using
val a = curriedAddTwo(10)(32)
val b = curriedAddTwo(13)(39)

a: Int = 42
b: Int = 52

In [5]:
// Currying in anonymous function syntax
val curriedAddTwoAnon: Int => Int => Int = (x: Int) => { (y:Int) => (x + y) } 

curriedAddTwoAnon: Int => Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$2008/0x0000000840a36840@22952087

In [6]:
val f1 = curriedAddTwoAnon(15) // Now this is closer to lettuce syntax
val f2 = curriedAddTwoAnon(15)(_) // This is also OK

f1: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$2020/0x0000000840a3d840@794b7cb1
f2: Int => Int = ammonite.$sess.cmd5$Helper$$Lambda$2021/0x0000000840a3e840@6f07400b

## Closures

A *closure* is a way to represent functions in our environments, containing the variable name, expression to be evaluated, and environment to execute the function in.

Why should a closure include an environment?

Consider the example:

~~~
let x = 5 in 
   let addx = function(y) x + y in 
      let x = 10 in 
          addx(10)
~~~

Or equivalently in scala:

~~~
val x = 5;
{
    val addx: Int => Int = (y: Int) => { x+y }
    {
        val x = 10;
        {
            addx(0)
        }
    }
}
~~~

What should this evaluate to?


In [6]:
// try out scala code after discussion

Most languages use static scoping, we want to do the same in Lettuce.

## Functions with Multiple Parameters

There are different ways to do this, we will explore one such way here, and you will explore another way in your homework.

Here, we will explore extending our closures to support multi parameter functions.

What other ways can we do this?
 - See autocurrying on the homework

For this exercise, we will use a small part of the grammer we have been working with and change our function calls and definitions.

$$
\begin{array}{c|c}
    \text{original grammer} & \text{updated multiparameter function grammer} \\\hline
    \begin{array}{rcl}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
    \end{array}
    &
    \begin{array}{rcll}
    \mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
    \mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
     & | & Ident(\mathbf{Identifier}) \\
     & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
     & | & FunDef( \mathbf{Identifier}^*, \mathbf{Expr}) & \text{Note multiple parameters now possible} \\ 
     & | & FunCall(\mathbf{Expr}, \mathbf{Expr}^*) & \text{function call - expr(expr1, ... , exprn)} \\
     & | & Let(\mathbf{Identifier},\mathbf{Expr}, \mathbf{Expr})  \\
    \end{array}
\end{array}
$$

Write the scala definition for `FunDef` and `FunCall`. Please use lists to implement the Kleene Star.

Note: Kleene star, represented by the '$^*$', is a way of representing multiple instances of a something.
It could be one, two, ten, or even none of the item in question.
For example, $\mathbf{Number}^*$ could be no numbers, a single number like $10$, or several numbers, like $10\ 21\ 32$.

In [7]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
//BEGIN SOLUTION
case class FunDef(idList: List[String], e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: List[Expr]) extends Expr
//END SOLUTION
case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Let
defined class FunDef
defined class FunCall
defined class TopLevel

We will now redefine closures for functions with zero or more args. When our function had one argument, our closures were defined as `Closure(id, expr, env)`. We would now like to define closures as
`Closure( [id1, ..., idn], expr, env)` where 
  - `id1..., idn` are the list of arguments for the function to be called. 
  - `expr` is the body of the function and 
  - `env` is the stored environment for static scoping.
 
 $$\begin{array}{rcl}
 \mathbf{Value} & \Rightarrow & Num(\mathbf{Double}) \\
 & \Rightarrow & Closure(\mathbf{String}^*, \mathbf{Expr}, \mathbf{Environment}) \\
 & \Rightarrow & Error \\
 \end{array}$$
 
 For __Environment__ please use a scala immutable map from __String__ to __Value__

In [8]:
sealed trait Value
case class Num(d: Double) extends Value
case object Error extends Value
//BEGIN SOLUTION
case class Closure(s: List[String], e: Expr, env: Map[String, Value]) extends Value
//END SOLUTION

defined trait Value
defined class Num
defined object Error
defined class Closure

Build an interpreter using the following semantic rules. Ensure that your interpreter correctly deals with the cases that give rise to error by throwing an exception.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array} (\text{#3})} $$
$$\newcommand\eval{\mathbf{eval}}$$
$$\semRule{}{\eval(\texttt{FunDef([id1,..., idk], e)},\sigma) = \text{Closure}(\texttt{[id1,..., idk]}, \texttt{e}, \sigma)}{fundef}$$

$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\forall\ i \in \{ 1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = v_i,  v_i \not= \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \eval(\texttt{fBody}, \color{red}{\sigma_{cl} \circ [id1 \rightarrow v_1, \ldots, idk \rightarrow v_k]})}{funcall-ok}$$
           
Let us interpret this rule.
   - __Purpose__ : Evaluate an expression of the form `FunCall(e, [e1,...,ek])` where `e` is the expr for the called function, and `e1, ..., ek` are exprs for the arguments of this call. There are $k$ arguments.
     - The called function `e` must evaluate to a closure of the form $\text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \sigma_{cl})$.
     - The function call must have the same number of arguments as the closure ($n=k$).
     - Each of the $k$ arguments `ei` must evaluate to a value $v_i$ where $v_i$ is not error.
     - Then the result of evaluating the function call is the same as that of evaluating `fBody` under the environment $\sigma_{cl}$ extended by mapping the formal parameters of the closure, `id1,.., idk`, to $v_1, \ldots, v_k$, respectively.


We can also write some error rules:

$$\semRule {\eval(\texttt{e}, \sigma) \not\in \text{Closure}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-not-a-function}$$


$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n \not= k}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-wrong-num-args}$$
           
  
$$\semRule {\eval(\texttt{e}, \sigma) = \text{Closure}(\texttt{[id1,..., idn]}, \texttt{fBody}, \color{red}{\sigma_{cl}}),\  \color{red}{n = k},\ (\exists\ i \in \{1, \ldots, k\})\ \eval(\texttt{ei}, \sigma) = \mathbf{error}}
           {\eval(\texttt{FunCall(e, [e1, ..., ek])}, \sigma) = \mathbf{error}}{funcall-arg-error}$$
           
           
**Hint:** Can map, zip, and foldLeft be used for this?
 

In [9]:
class ErrorException(s:String) extends Exception(s){}  


def valueToNumber(v: Value): Double = v match {
    case Num(d) => d
    case _ => throw new ErrorException(s"Could not convert value $v to a number")
}

def eval(e: Expr, env: Map[String, Value]): Value = {
    def addValues(v1: Value, v2: Value): Value = 
        Num ( valueToNumber(v1) + valueToNumber(v2) )
    
    e match {
        case Const(d) => Num(d)
        case Ident(x) => {
            if (env contains x) {
                env(x)
            } else {
                throw new ErrorException(s"Could not find identifier $x")
            }
        }
        case Plus(e1,e2) => {
            val v1 = eval(e1, env)
            val v2 = eval(e2, env)
            addValues(v1,v2)
        }
        case Let(id, e1, e2) => {
            val v1 = eval(e1, env)
            val env2 = env + (id -> v1)
            eval(e2, env2)
        }
        
        case FunDef(idList, e) => {
            Closure(idList, e, env)
        }
        
        /* the single parameter version of FunCall
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            // Since evaluating e2 did not fail with an exception,
            // if I reach this point in my execution, I know that
            // neither v1 nor v2 are error.
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        // */
        
        case FunCall(e, eList) => {
            //BEGIN SOLUTION
            val v = eval(e, env)
            v match {
                case Closure(argList, fBody, closureEnv) => {
                    if (argList.length == eList.length){
                        val valList = eList.map {eval(_, env)}
                        val zipList = argList.zip(valList)
                        val newMap = zipList.foldLeft[Map[String, Value]] (env) { (newEnv, newBind) => newEnv + newBind }
                        eval(fBody, newMap)
                    } else {
                        throw new ErrorException(s"Argument length mismatch")
                    }
                }
                case _ => throw new ErrorException(s"Tried to use a numerical value as a function")
            }
            //END SOLUTION
        }
    }
}


def evalProgram(p: Program): Value = p match {
    case TopLevel(e) => try
            eval(e, Map.empty)
    catch {
        case e: ErrorException => {
            println(e)
            Error
        }
        case e: IllegalArgumentException => {
            println(e)
            Error
        }
        case e => {
            println("Unknown Exception " + e.toString)
            Error
        }
            
    }
}


defined class ErrorException
defined function valueToNumber
defined function eval
defined function evalProgram

### Tests

In [10]:
//BEGIN TEST
// TEST three arguments
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo2")
// function(x, y,z)  x + y+z+z
val fun2 = FunDef(List("x", "y", "z"), Plus(Plus(Plus(x, y), z), z))
//let foo = function(x,y,z) x+y+z+z in foo(10, 20, 30)
val l2 = Let("foo2", fun2, FunCall(foo, List(Const(10.0), Const(20.0), Const(30.0))))
//Program
val p2 = TopLevel(l2)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(90.0), "Test 2 Failed")
//END TEST

Your program evaluated to Num(90.0)


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
foo: Ident = Ident("foo2")
fun2: FunDef = FunDef(
  List("x", "y", "z"),
  Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("z")), Ident("z"))
)
l2: Let = Let(
  "foo2",
  FunDef(
    List("x", "y", "z"),
    Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("z")), Ident("z"))
  ),
  FunCall(Ident("foo2"), List(Const(10.0), Const(20.0), Const(30.0)))
)
p2: TopLevel = TopLevel(
  Let(
    "foo2",
    FunDef(
      List("x", "y", "z"),
      Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("z")), Ident("z"))
    ),
    FunCall(Ident("foo2"), List(Const(10.0), Const(20.0), Const(30.0)))
  )
)
v2: Value = Num(90.0)

In [11]:
//BEGIN TEST
//Test zero arguments
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List())
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p2 = TopLevel(l3)
val v3 = evalProgram(p2)
println(s"Your program evaluated to $v3")
assert(v3 == Num(10.0), "Test 3 Failed")
//END TEST

Your program evaluated to Num(10.0)


x: Ident = Ident("x")
foo: Ident = Ident("foo")
fcall: FunCall = FunCall(Ident("foo"), List())
fdef: FunDef = FunDef(List(), Ident("x"))
l2: Let = Let("foo", FunDef(List(), Ident("x")), FunCall(Ident("foo"), List()))
l3: Let = Let(
  "x",
  Const(10.0),
  Let("foo", FunDef(List(), Ident("x")), FunCall(Ident("foo"), List()))
)
p2: TopLevel = TopLevel(
  Let(
    "x",
    Const(10.0),
    Let("foo", FunDef(List(), Ident("x")), FunCall(Ident("foo"), List()))
  )
)
v3: Value = Num(10.0)

In [12]:
//BEGIN TEST
//Evaluate with wrong number of args
val x = Ident("x")
val foo = Ident("foo")
val fcall = FunCall(foo, List(Const(1.0)))
val fdef = FunDef(List(), x)
val l2 = Let("foo", fdef, fcall)
val l3 = Let("x", Const(10), l2)
val p4 = TopLevel(l3)
val v4 = evalProgram(p4)
assert(v4 == Error, "Test 4 failed -- your program should have detected that arguments were mismatched")
//END TEST

ammonite.$sess.cmd8$Helper$ErrorException: Argument length mismatch


x: Ident = Ident("x")
foo: Ident = Ident("foo")
fcall: FunCall = FunCall(Ident("foo"), List(Const(1.0)))
fdef: FunDef = FunDef(List(), Ident("x"))
l2: Let = Let(
  "foo",
  FunDef(List(), Ident("x")),
  FunCall(Ident("foo"), List(Const(1.0)))
)
l3: Let = Let(
  "x",
  Const(10.0),
  Let(
    "foo",
    FunDef(List(), Ident("x")),
    FunCall(Ident("foo"), List(Const(1.0)))
  )
)
p4: TopLevel = TopLevel(
  Let(
    "x",
    Const(10.0),
    Let(
      "foo",
      FunDef(List(), Ident("x")),
      FunCall(Ident("foo"), List(Const(1.0)))
    )
  )
)
v4: Value = Error

In [13]:
//BEGIN TEST
// TEST scoping
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val foo = Ident("foo")

// function(x, y)  x + y + x + z 
val fun = FunDef(List("x", "y"), Plus(Plus(Plus(x, y), x),z))
val funcall = FunCall(foo, List(Const(10.0), Const(30.0)))
// let x = 1 in
//     let z = 50 in
//       let foo = fun(x,y) x + y + x + z in
//         let = 2 in
//           f(10,30)
val l4 = Let("x", Const(1.0), Let("z", Const(50.0), Let("foo", fun, Let("y", Const(2.0), funcall))))
//Program
val p2 = TopLevel(l4)
val v2 = evalProgram(p2)
println(s"Your program evaluated to $v2")
assert(v2 == Num(100.0), "Test 5 Failed == you did not evaluate static scoping correctly")
//END TEST

Your program evaluated to Num(100.0)


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
foo: Ident = Ident("foo")
fun: FunDef = FunDef(
  List("x", "y"),
  Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("x")), Ident("z"))
)
funcall: FunCall = FunCall(Ident("foo"), List(Const(10.0), Const(30.0)))
l4: Let = Let(
  "x",
  Const(1.0),
  Let(
    "z",
    Const(50.0),
    Let(
      "foo",
      FunDef(
        List("x", "y"),
        Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("x")), Ident("z"))
      ),
      Let(
        "y",
        Const(2.0),
        FunCall(Ident("foo"), List(Const(10.0), Const(30.0)))
      )
    )
  )
)
p2: TopLevel = TopLevel(
  Let(
    "x",
    Const(1.0),
    Let(
      "z",
      Const(50.0),
      Let(
        "foo",
        FunDef(
          List("x", "y"),
          Plus(Plus(Plus(Ident("x"), Ident("y")), Ident("x")), Ident("z"))
        ),
        Let(
          "y",
          Const(2.0),
          FunCall(Ident("foo"), List(Const(10.0), Const(30.0)))
        )
      )
    )
 

## Thinking back to Recursion

What happens when we want to do recursion? What goes wrong?

What solution did we arrive at in class?